# Setup

In [ ]:
!pip install tensorboardX --quiet
!pip install tensorboard --quiet


In [ ]:
# Install deps
! pip install fairseq sacrebleu sentencepiece --quiet


In [ ]:
# Install nvidia apex for multi-tensor support
# Ref: https://zhuanlan.zhihu.com/p/194176917
%cd /home/ubuntu/nmt
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

import fairseq
print(fairseq.utils.multi_tensor_l2norm_available)


## Downloads

In [ ]:
# FLoRes repo clone
%cd /home/ubuntu/nmt
!git clone https://github.com/facebookresearch/flores.git

In [ ]:
# get mBART pretrained model
%cd /home/ubuntu/nmt/flores/
!wget https://dl.fbaipublicfiles.com/fairseq/models/mbart/mbart.cc25.v2.tar.gz
!tar -xzvf mbart.cc25.v2.tar.gz
!rm -rf mbart.cc25.v2.tar.gz

# Focal Loss setup
Note: Make sure to upload the focal_loss.py file to notebook home

In [ ]:
%cd /home/ubuntu/nmt/
!cp -rf focal_loss.py $(pip show fairseq | grep Location | cut -d' ' -f2)/fairseq/criterions/

# CodeEnv Setup

In [ ]:
SRC="si"
PARENT_SRC="ta"
TGT="en"
BPE=6 #6 for si-en
NUM_UPDATES=45000
SAVE_INTERVAL=15000

In [ ]:
%cd /home/ubuntu/nmt/
!cp -f download_data.sh ./flores/download_data.sh
!cp -f prepare_parallel.sh ./flores/prepare_parallel.sh
!cp -f prepare_reverse.sh ./flores/prepare_reverse.sh


## Data Setup

In [ ]:
# Training data
%cd /home/ubuntu/nmt/flores/
!bash download-data.sh --langs $SRC"-"$TGT

In [ ]:
%cd /home/ubuntu/nmt/flores/
! bash prepare_parallel.sh --src $PARENT_SRC --tgt $TGT --bpe $BPE

In [ ]:
%cd /home/ubuntu/nmt/flores/
! bash prepare_parallel.sh --src $SRC --tgt $TGT --bpe $BPE

# Fairseq-train

In [ ]:
#########################################################################################################
# Transfer Learning - Parent lang
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/"wiki_"$PARENT_SRC"_"$TGT"_bpe5000/" \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang $PARENT_SRC --target-lang $TGT \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update $NUM_UPDATES \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file mbart.cc25.v2/model.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir "TensorboardParent"$PARENT_SRC$TGT/ \
  --max-update $NUM_UPDATES \
  --checkpoint-suffix "_parent_"$PARENT_SRC"_"$TGT \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a "mbart-moses-"$PARENT_SRC"-"$TGT"-screenlog"


In [ ]:
#########################################################################################################
# Transfer Learning - (FineTune) Child lang
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/"wiki_"$SRC"_"$TGT"_bpe5000/" \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang $SRC --target-lang $TGT \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update $NUM_UPDATES \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file /home/ubuntu/nmt/flores/checkpoints/"checkpoint_best_parent_"$TGT"_"$SRC".pt" \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir "TensorboardChild"$SRC$TGT/ \
  --max-update $NUM_UPDATES \
  --checkpoint-suffix "_tl_finetune_"$SRC"_"$TGT \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a "mbart-moses-"$SRC"-"$TGT"-screenlog"


In [ ]:
#########################################################################################################
# Transfer Learning - Parent lang
#########################################################################################################
# TA-EN
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/wiki_ta_en_bpe5000/ \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang ta --target-lang en \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file mbart.cc25.v2/model.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir TensorboardParentTAEN/ \
  --max-update 40000 \
  --checkpoint-suffix "_parent_ta_en" \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a mbart-moses-ta-en-screenlog
#########################################################################################################
# HI-EN
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/wiki_hi_en_bpe5000/ \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang hi --target-lang en \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file mbart.cc25.v2/model.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir TensorboardParentHIEN/ \
  --max-update 40000 \
  --checkpoint-suffix "_parent_hi_en" \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a mbart-moses-hi-en-screenlog
#########################################################################################################
# FA-EN
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/wiki_fa_en_bpe5000/ \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang fa --target-lang en \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file mbart.cc25.v2/model.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir TensorboardParentFAEN/ \
  --max-update 40000 \
  --checkpoint-suffix "_parent_fa_en" \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a mbart-moses-fa-en-screenlog
#########################################################################################################
# VI-EN
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/wiki_vi_en_bpe5000/ \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang vi --target-lang en \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file mbart.cc25.v2/model.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir TensorboardParentVIEN/ \
  --max-update 40000 \
  --checkpoint-suffix "_parent_vi_en" \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a mbart-moses-vi-en-screenlog

#########################################################################################################
# Transfer Learning - Child lang
#########################################################################################################
# SI-EN
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/wiki_si_en_bpe5000/ \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang si --target-lang en \
  --criterion focal_loss\
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file /home/ubuntu/nmt/flores/checkpoints/checkpoint_best_parent_ta_en.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir TensorboardChildSIEN/ \
  --max-update 40000 \
  --checkpoint-suffix "_tl_finetune_si_en" \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a mbart-moses-si-en-screenlog
#########################################################################################################
# NE-EN
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/wiki_ne_en_bpe5000/ \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang ne --target-lang en \
  --criterion focal_loss \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file /home/ubuntu/nmt/flores/checkpoints/checkpoint_best_parent_hi_en.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir TensorboardChildNEEN/ \
  --max-update 40000 \
  --checkpoint-suffix "_tl_finetune_ne_en" \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a mbart-moses-ne-en-screenlog
#########################################################################################################
# PS-EN
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/wiki_ps_en_bpe5000/ \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang ps --target-lang en \
  --criterion focal_loss \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file /home/ubuntu/nmt/flores/checkpoints/checkpoint_best_parent_fa_en.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir TensorboardChildPSEN/ \
  --max-update 40000 \
  --checkpoint-suffix "_tl_finetune_ps_en" \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a mbart-moses-ps-en-screenlog
#########################################################################################################
# KM-EN
#########################################################################################################
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
! fairseq-train ./data-bin/wiki_km_en_bpe5000/ \
  --encoder-normalize-before --decoder-normalize-before \
  --arch mbart_large --layernorm-embedding \
  --task translation_from_pretrained_bart \
  --source-lang km --target-lang en \
  --criterion focal_loss \
  --optimizer adam --adam-eps 1e-06 --adam-betas '(0.9, 0.98)' \
  --lr-scheduler polynomial_decay --lr 3e-05 --warmup-updates 2500 --total-num-update 40000 \
  --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
  --max-tokens 1024 --update-freq 2 \
  --save-interval 1 --no-epoch-checkpoints --skip-invalid-size-inputs-valid-test \
  --seed 222 --log-format json --log-interval 2 \
  --restore-file /home/ubuntu/nmt/flores/checkpoints/checkpoint_best_parent_vi_en.pt \
  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
  --langs $langs \
  --tensorboard-logdir TensorboardChildKMEN/ \
  --max-update 40000 \
  --checkpoint-suffix "_tl_finetune_km_en" \
  --ddp-backend c10d --memory-efficient-fp16 |& tee -a mbart-moses-km-en-screenlog


# Eval

In [ ]:
#Generate Sacrebleu
langs='ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN'
%cd /home/ubuntu/nmt/flores/
!fairseq-generate ./data-bin/"wiki_"$SRC"_"$TGT"_bpe5000/" \
    --source-lang $SRC --target-lang $TGT \
    --path /home/ubuntu/nmt/flores/checkpoints/"checkpoint_best_tl_finetune_"$SRC"_"$TGT".pt" \
    --task translation_from_pretrained_bart \
    --lenpen 1.2 \
    --gen-subset test \
    --bpe 'sentencepiece' --sentencepiece-model mbart.cc25.v2/sentence.bpe.model \
    --scoring sacrebleu  --batch-size 32 --langs $langs > $SRC"_"$TGT"_bleu_w_bpe"

!tail -1 $SRC"_"$TGT"_bleu_w_bpe"


In [ ]:
# !cat $SRC"_"$TGT"_bleu_w_bpe" | grep -P "^D" |sort -V |cut -f 3- | sed 's/\[en_XX\]//g' > $SRC"_"$TGT".hyp"
# !cat $SRC"_"$TGT"_bleu_w_bpe" | grep -P "^T" |sort -V |cut -f 2- | sed 's/\[en_XX\]//g' > $SRC"_"$TGT".ref"
# import sacrebleu
# my_hyp_file = open($SRC"_"$TGT".hyp", "r")
# hyp = my_hyp_file.readlines()
# my_ref_file = open($SRC"_"$TGT".ref", "r")
# ref_tmp = my_ref_file.readlines()
# ref = [sent for sent in ref_tmp]
# ref =[ref]
# bleu = sacrebleu.corpus_bleu(hyp,ref)
# print(bleu.score)

# Housekeeping

In [ ]:
# Upload checkpoint
!aws s3 cp /home/ubuntu/nmt/flores/checkpoints/"checkpoint_best_parent_"$PARENT_SRC"_"$TGT".pt" s3://msainmtntu/checkpoints/"checkpoint_best_parent_"$PARENT_SRC"_"$TGT".pt"
!rm -rf /home/ubuntu/nmt/flores/checkpoints/*"0000_parent_"$PARENT_SRC"_"$TGT".pt"


In [ ]:
# Upload checkpoint
!aws s3 cp /home/ubuntu/nmt/flores/checkpoints/"checkpoint_best_tl_finetune_"$SRC"_"$TGT".pt" s3://msainmtntu/checkpoints/"checkpoint_best_tl_finetune_"$SRC"_"$TGT".pt"
!rm -rf /home/ubuntu/nmt/flores/checkpoints/*"0000_tl_finetune_"$SRC"_"$TGT".pt"


In [ ]:
# Upload tensorboard
# !aws s3 cp --recursive /home/ubuntu/nmt/flores/Tensorboard s3://msainmtntu/Tensorboards/TensorboardBaseline$(echo "$SRC" | tr a-z A-Z)$(echo "$TGT" | tr a-z A-Z)
!aws s3 cp --recursive /home/ubuntu/nmt/flores/TensorboardParent$(echo "$PARENT_SRC" | tr a-z A-Z)$(echo "$TGT" | tr a-z A-Z) s3://msainmtntu/Tensorboards/TensorboardParent$(echo "$PARENT_SRC" | tr a-z A-Z)$(echo "$TGT" | tr a-z A-Z)
!aws s3 cp --recursive /home/ubuntu/nmt/flores/TensorboardChild$(echo "$SRC" | tr a-z A-Z)$(echo "$TGT" | tr a-z A-Z) s3://msainmtntu/Tensorboards/TensorboardChild$(echo "$SRC" | tr a-z A-Z)$(echo "$TGT" | tr a-z A-Z)


## Checks

In [ ]:
!ls -lh /home/ubuntu/nmt/flores/checkpoints/

In [ ]:
!tail -5 /home/ubuntu/nmt/flores/"mbart-moses-"$PARENT_SRC"-"$TGT"-screenlog"
!date

In [ ]:
!tail -5 /home/ubuntu/nmt/flores/"mbart-moses-"$SRC"-"$TGT"-screenlog"
!date